# WFI Reference File Pipeline - ReadNoise Example

This notebook shows how to use the WFI Reference File Pipeline (RFP) to create readnoise reference files. The RFP will simulate dark reads that will be used as input into the ReadNoise module. 

Additional functionality such as reading a list of files to construct the input data, and optional methods to calculate the read noise are in development and only mentioned, but are not necessary to generate reference files suitable for delivery to CRDS.

## Imports

The imports here are used in the below cells.  
NOTE: We are configuring logging which will require you to have a config.yml file in the /src/wfi_reference_pipeline/config directory.

In [18]:
import os
import logging
from wfi_reference_pipeline.utilities.simulate_reads import simulate_dark_reads
from wfi_reference_pipeline.resources.make_dev_meta import MakeDevMeta
from wfi_reference_pipeline.reference_types.readnoise.readnoise import ReadNoise
from wfi_reference_pipeline.constants import WFI_MODE_WIM, WFI_FRAME_TIME, REF_TYPE_READNOISE
from wfi_reference_pipeline.utilities import logging_functions
logging_functions.configure_logging("readnoise_example")

'/grp/roman/rcosenti/rfp_pipeline_testing/logs/DEV_readnoise_example_rcosenti_2024-04-22-17-27.log'

## Setup 

The example read noise reference file will be created in the RFP scratch directory. This directory has it contents automatically deleted every night. A default outfile name is given and if clobber set to True will be overwritten.

In [3]:
write_path = '/grp/roman/RFP/DEV/scratch/'  # Set the write path to be in the RFP scratch directory.
outfile = write_path + 'roman_dev_readnoise.asdf'  # Default read noise development filename.

## Meta Data

The RFP utilizes python data classes to construct reference file type dependent meta data for development from the "ref_type" keyword, which is converted into a dictionary when populating the data model.

In [4]:
logging.info("Generating Readnoise Metadata")
tmp = MakeDevMeta(ref_type=REF_TYPE_READNOISE)  # Get the default reference file type specific development meta data.

In [7]:
print(tmp.meta_readnoise)  # Inspect default development meta data.

WFIMetaReadNoise(reference_type='READNOISE', pedigree='DUMMY', description='For RFP Development and DMS Build Support.', author='RFP', use_after=<Time object: scale='utc' format='isot' value=2023-01-01T00:00:00.000>, telescope='ROMAN', origin='STSCI', instrument='WFI', instrument_detector='WFI01')


In [15]:
det = 5  # Make the readnoise reference file for WFI05.
tmp.meta_readnoise.instrument = {'detector': f'WFI{det:02d}'}
tmp.meta_readnoise.useafter = '2024-04-22T00:00:00.000'  # Update useafter to the new string
tmp.meta_readnoise

WFIMetaReadNoise(reference_type='READNOISE', pedigree='DUMMY', description='For RFP Development and DMS Build Support.', author='RFP', use_after=<Time object: scale='utc' format='isot' value=2023-01-01T00:00:00.000>, telescope='ROMAN', origin='STSCI', instrument={'detector': 'WFI05'}, instrument_detector='WFI01')

In [16]:
print(tmp.meta_readnoise)  # Inspect updated meta data.

WFIMetaReadNoise(reference_type='READNOISE', pedigree='DUMMY', description='For RFP Development and DMS Build Support.', author='RFP', use_after=<Time object: scale='utc' format='isot' value=2023-01-01T00:00:00.000>, telescope='ROMAN', origin='STSCI', instrument={'detector': 'WFI05'}, instrument_detector='WFI01')


## Simulate Data

Use the RFP simulate reads function to create a cube of dark reads with input parameters. The current assumed input to create the read noise reference file is a series of dark reads. The read noise reference file is WFI mode specific and therefore CRDS requires that there are separate WFI Imaging Mode (WIM) or WFI Spetral Mode (WSM) reference files available for the WFI optical element filters and WFI spectral components GRISM and PRISM.

In [21]:
tot_reads = 10  # The number of reads in the simulated cube.

# We are using the WFI_MODE_WIM which has a frame time of 3.04 seconds for each read to simulate the data cube.
sim_read_cube, _ = simulate_dark_reads(tot_reads)

Making WFI Imaging Mode (WIM) dark read cube with exposure time 3.04 seconds.


## Instantiate RFP readnoise

The reference file pipeline has a reference file base class which when any reference file type, like ReadNoise here, is instantiated it inherits the base class attributes and methods. The reference file base class has positional arguments and it is necessary to declare the data, the first positional input to the reference file base class, as None to properly inherit the reference file base class attributes. If None is replaced by a file or list of files, ReadNoise will follow a different flow to create input data to generate a reference file. 

In [23]:
RFPreadnoise = ReadNoise(None,
                         meta_data=tmp.meta_readnoise,
                         outfile=outfile,
                         clobber=True,
                         user_input=sim_read_cube)

TypeError: ReadNoise.__init__() got an unexpected keyword argument 'user_input'

## Compute Readnoise

The readnoise is determined by fitting a linear ramp to the input data cube. The variance of the residuals from the ramp is set by the ReadNoise module to be the readnoise "data" in the asdf file datamodel. 

In [9]:
RFPreadnoise.comp_ramp_res_var()  # Compute the variance of the residuals in the fitted ramp.

## Other ReadNoise Calculations

Other methods are available for use, such as computing the correlated-double-sampling (CDS) noise. Such methods could be used to compare the implemented algorithm in the RFP and its performance as compared to another method. See RFP documentation.

## Build the datamodel for the reference file, write it to disk, and change the permissions of the file. 

Other methods are available to determine the readnoise in line with what others have called the "total noise" or correlated-double-sampling noise (CDS noise). See RFP documentation.

In [11]:
RFPreadnoise.save_outfile()  # Save the readnoise object into the data model
print('Made file -> ', outfile)
logging.info(f'outfile created: {outfile}')

Made file ->  /grp/roman/RFP/DEV/scratch/roman_dev_readnoise.asdf
